In [22]:
#import necessary library
import pandas as pd, numpy as np, re, unicodedata, hashlib, string
from IPython.display import HTML
import base64

In [23]:
#import dataset
path = "https://raw.githubusercontent.com/the-buike/Business-Health-Growth-Analysis/refs/heads/main/Inventory_SupplyChain_Dataset.csv"


In [24]:
#Load the data
# We try to parse the 'Date' column as real dates.
inventory = pd.read_csv(path, low_memory=False)

In [25]:
#Data profile check
def basic_profile(df: pd.DataFrame, top_n: int = 5) -> dict:
    """Return a dict with basic dataset info."""
    info = {}

    # Size and columns
    info["shape"] = df.shape
    info["n_rows"] = len(df)
    info["n_cols"] = df.shape[1]
    info["columns"] = list(df.columns)

    # Memory
    info["memory_mb"] = round(df.memory_usage(deep=True).sum() / 1_048_576, 3)

    # Dtypes per column
    info["dtypes"] = df.dtypes.astype(str).to_dict()

    # Missing per column
    miss_cnt = df.isna().sum()
    miss_pct = (miss_cnt / len(df)).round(4)
    info["missing_per_column"] = (
        pd.DataFrame({"missing_count": miss_cnt, "missing_pct": miss_pct})
        .sort_values("missing_pct", ascending=False)
    )

    # Unique counts per column
    info["unique_per_column"] = (
        df.nunique(dropna=False)
        .rename("unique_values")
        .to_frame()
        .sort_values("unique_values", ascending=False)
    )

    # Sample rows
    info["head"] = df.head(top_n)
    info["tail"] = df.tail(top_n)

    # Quick stats
    info["describe_numeric"] = df.select_dtypes(include=[np.number]).describe().T
    info["describe_categorical"] = (
        df.select_dtypes(exclude=[np.number])
          .agg(["nunique"])
          .T
          .rename(columns={"nunique": "unique_values"})
    )

    return info

def print_basic_profile(profile: dict, max_rows: int = 10):
    print(f"Shape: {profile['shape']}  |  Rows: {profile['n_rows']}  |  Cols: {profile['n_cols']}")
    print(f"Memory: {profile['memory_mb']} MB")
    print("\nDtypes:")
    for c, t in profile["dtypes"].items():
        print(f"  {c}: {t}")

    print("\nMissing per column (top):")
    print(profile["missing_per_column"].head(max_rows))

    print("\nUnique per column (top):")
    print(profile["unique_per_column"].head(max_rows))

    print("\nNumeric summary:")
    if not profile["describe_numeric"].empty:
        print(profile["describe_numeric"].head(max_rows))
    else:
        print("  No numeric columns")

    print("\nCategorical summary:")
    if not profile["describe_categorical"].empty:
        print(profile["describe_categorical"].head(max_rows))
    else:
        print("  No categorical columns")

    print("\nHead:")
    print(profile["head"])
    print("\nTail:")
    print(profile["tail"])

In [26]:
#check
profile = basic_profile(inventory)
print_basic_profile(profile)

Shape: (1200, 15)  |  Rows: 1200  |  Cols: 15
Memory: 0.464 MB

Dtypes:
  Date: object
  Region: object
  Category: object
  Supplier: object
  Warehouse: object
  Order Status: object
  Units Sold: int64
  Inventory Level: int64
  Transportation Cost: float64
  Order Accuracy: bool
  Lead Time (Days): int64
  Backorder: bool
  Cost of Goods Sold (COGS): float64
  Average Inventory: float64
  Warehouse Capacity: int64

Missing per column (top):
                     missing_count  missing_pct
Date                             0          0.0
Region                           0          0.0
Category                         0          0.0
Supplier                         0          0.0
Warehouse                        0          0.0
Order Status                     0          0.0
Units Sold                       0          0.0
Inventory Level                  0          0.0
Transportation Cost              0          0.0
Order Accuracy                   0          0.0

Unique per column (top

In [27]:
#missing values
def missing_summary(df: pd.DataFrame) -> dict:
    """High-level missing data summary."""
    total_cells = df.shape[0] * df.shape[1]
    total_missing = int(df.isna().sum().sum())
    return {
        "rows": df.shape[0],
        "cols": df.shape[1],
        "total_cells": total_cells,
        "total_missing": total_missing,
        "pct_missing_overall": round(100 * total_missing / total_cells, 2) if total_cells else 0.0,
        "rows_with_any_missing": int(df.isna().any(axis=1).sum()),
        "cols_with_any_missing": int(df.isna().any(axis=0).sum()),
        "cols_all_missing": df.columns[df.isna().all()].tolist()
    }

def missing_report(df: pd.DataFrame, sort_by: str = "pct", descending: bool = True) -> pd.DataFrame:
    """
    Per-column missing report.
    sort_by: "pct" or "count"
    """
    m_count = df.isna().sum()
    m_pct = (m_count / len(df) * 100).round(2) if len(df) else 0
    rep = pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "non_nulls": df.notna().sum(),
        "missing_count": m_count,
        "missing_pct": m_pct
    })
    key = "missing_pct" if sort_by == "pct" else "missing_count"
    return rep.sort_values(key, ascending=not descending)

def missing_flags(df: pd.DataFrame, high_thresh: float = 50.0) -> dict:
    """Quick flags for columns above a threshold of missingness."""
    pct = (df.isna().sum() / len(df) * 100) if len(df) else pd.Series(0, index=df.columns)
    return {
        "high_missing_cols": pct[pct >= high_thresh].sort_values(ascending=False).round(2).to_dict(),
        "low_missing_cols": pct[(pct > 0) & (pct < high_thresh)].sort_values(ascending=False).round(2).to_dict(),
        "no_missing_cols": pct[pct == 0].index.tolist()
    }

In [28]:
# Overall view
summary = missing_summary(inventory)
summary

{'rows': 1200,
 'cols': 15,
 'total_cells': 18000,
 'total_missing': 0,
 'pct_missing_overall': 0.0,
 'rows_with_any_missing': 0,
 'cols_with_any_missing': 0,
 'cols_all_missing': []}

In [29]:
# Column-level table
report = missing_report(inventory, sort_by="pct")
report.head(20)

,dtype,non_nulls,missing_count,missing_pct
Date,object,1200,0,0.0
Region,object,1200,0,0.0
Category,object,1200,0,0.0
Supplier,object,1200,0,0.0
Warehouse,object,1200,0,0.0
Order Status,object,1200,0,0.0
Units Sold,int64,1200,0,0.0
Inventory Level,int64,1200,0,0.0
Transportation Cost,float64,1200,0,0.0
Order Accuracy,bool,1200,0,0.0


In [30]:
# Flag columns above 50% missing
flags = missing_flags(inventory, high_thresh=50)
flags

{'high_missing_cols': {},
 'low_missing_cols': {},
 'no_missing_cols': ['Date',
  'Region',
  'Category',
  'Supplier',
  'Warehouse',
  'Order Status',
  'Units Sold',
  'Inventory Level',
  'Transportation Cost',
  'Order Accuracy',
  'Lead Time (Days)',
  'Backorder',
  'Cost of Goods Sold (COGS)',
  'Average Inventory',
  'Warehouse Capacity']}

In [31]:
#check pr nulls and nans etc
def unify_missing_tokens(df, tokens=("","na","n/a","null","none","nan","missing","?")):
    out = df.copy()
    toks = set(t.lower() for t in tokens)
    for c in out.columns:
        if out[c].dtype == "object" or pd.api.types.is_string_dtype(out[c]):
            s = out[c].astype("string")
            out[c] = s.where(~s.str.lower().isin(toks), other=pd.NA)
    return out

In [32]:
df = unify_missing_tokens(inventory)

In [33]:
df

,Date,Region,Category,Supplier,Warehouse,Order Status,Units Sold,Inventory Level,Transportation Cost,Order Accuracy,Lead Time (Days),Backorder,Cost of Goods Sold (COGS),Average Inventory,Warehouse Capacity
0,14-01-2020,North,Accessories,Supplier A,Warehouse 1,Fulfilled,302,2124,1103.838324,True,9,False,37820.05240,2048.0,5037
1,15-11-2020,East,Furniture,Supplier D,Warehouse 2,Fulfilled,741,1972,13163.007660,True,11,False,54396.17369,1213.0,9216
2,16-04-2020,East,Furniture,Supplier C,Warehouse 2,Fulfilled,940,454,9872.294126,True,17,False,24217.45462,1160.5,7699
3,17-04-2020,South,Accessories,Supplier D,Warehouse 2,Canceled,589,1867,4547.589932,True,3,False,38919.08005,3364.5,9271
4,18-04-2020,North,Accessories,Supplier C,Warehouse 2,Fulfilled,964,4862,11994.042310,True,21,False,59204.50683,3063.0,5828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,05-01-2023,East,Electronics,Supplier C,Warehouse 2,Pending,488,4489,13060.376620,True,29,False,64393.19690,3614.0,8820
1196,04-01-2023,East,Clothing,Supplier A,Warehouse 3,Fulfilled,946,2739,3052.358948,True,14,False,105773.05570,2015.0,7697
1197,03-01-2023,North,Accessories,Supplier C,Warehouse 2,Fulfilled,637,1291,3774.674667,True,15,False,96050.32391,758.5,5386
1198,02-01-2023,West,Clothing,Supplier D,Warehouse 1,Pending,906,226,11776.903590,True,2,False,88010.31702,1761.5,8729


In [34]:
summary

{'rows': 1200,
 'cols': 15,
 'total_cells': 18000,
 'total_missing': 0,
 'pct_missing_overall': 0.0,
 'rows_with_any_missing': 0,
 'cols_with_any_missing': 0,
 'cols_all_missing': []}

In [35]:
report

,dtype,non_nulls,missing_count,missing_pct
Date,object,1200,0,0.0
Region,object,1200,0,0.0
Category,object,1200,0,0.0
Supplier,object,1200,0,0.0
Warehouse,object,1200,0,0.0
Order Status,object,1200,0,0.0
Units Sold,int64,1200,0,0.0
Inventory Level,int64,1200,0,0.0
Transportation Cost,float64,1200,0,0.0
Order Accuracy,bool,1200,0,0.0


In [37]:
#standardize columns
# define once
import re, unicodedata, pandas as pd

def to_snake(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii")
    s = re.sub(r"(?<=[a-z0-9])(?=[A-Z])", "_", s)     # split CamelCase
    s = re.sub(r"[^\w]+", "_", s)                     # non-alnum -> _
    s = re.sub(r"_+", "_", s).strip("_").lower()
    if s and s[0].isdigit():
        s = "_" + s
    return s or "col"

def snakecase_columns(df: pd.DataFrame, inplace: bool = False) -> pd.DataFrame:
    cols = [to_snake(c) for c in df.columns]
    # ensure uniqueness after cleaning
    seen, unique = {}, []
    for c in cols:
        if c in seen:
            seen[c] += 1
            unique.append(f"{c}_{seen[c]}")
        else:
            seen[c] = 0
            unique.append(c)
    if inplace:
        df.columns = unique
        return df
    out = df.copy()
    out.columns = unique
    return out


In [38]:
snakecase_columns(inventory, inplace=True)

,date,region,category,supplier,warehouse,order_status,units_sold,inventory_level,transportation_cost,order_accuracy,lead_time_days,backorder,cost_of_goods_sold_cogs,average_inventory,warehouse_capacity
0,14-01-2020,North,Accessories,Supplier A,Warehouse 1,Fulfilled,302,2124,1103.838324,True,9,False,37820.05240,2048.0,5037
1,15-11-2020,East,Furniture,Supplier D,Warehouse 2,Fulfilled,741,1972,13163.007660,True,11,False,54396.17369,1213.0,9216
2,16-04-2020,East,Furniture,Supplier C,Warehouse 2,Fulfilled,940,454,9872.294126,True,17,False,24217.45462,1160.5,7699
3,17-04-2020,South,Accessories,Supplier D,Warehouse 2,Canceled,589,1867,4547.589932,True,3,False,38919.08005,3364.5,9271
4,18-04-2020,North,Accessories,Supplier C,Warehouse 2,Fulfilled,964,4862,11994.042310,True,21,False,59204.50683,3063.0,5828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,05-01-2023,East,Electronics,Supplier C,Warehouse 2,Pending,488,4489,13060.376620,True,29,False,64393.19690,3614.0,8820
1196,04-01-2023,East,Clothing,Supplier A,Warehouse 3,Fulfilled,946,2739,3052.358948,True,14,False,105773.05570,2015.0,7697
1197,03-01-2023,North,Accessories,Supplier C,Warehouse 2,Fulfilled,637,1291,3774.674667,True,15,False,96050.32391,758.5,5386
1198,02-01-2023,West,Clothing,Supplier D,Warehouse 1,Pending,906,226,11776.903590,True,2,False,88010.31702,1761.5,8729


In [39]:
#check datatype columns
def dtype_report(df: pd.DataFrame) -> pd.DataFrame:
    """
    Report basic info about each column's data type.
    """
    rep = pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "non_nulls": df.notna().sum(),
        "nulls": df.isna().sum(),
        "unique_values": df.nunique(dropna=True)
    })
    rep["is_numeric"] = rep["dtype"].isin(["int64", "float64"])
    rep["is_object_or_string"] = rep["dtype"].isin(["object", "string"])
    return rep

In [40]:
dtype_report(inventory)

,dtype,non_nulls,nulls,unique_values,is_numeric,is_object_or_string
date,object,1200,0,1200,False,True
region,object,1200,0,4,False,True
category,object,1200,0,4,False,True
supplier,object,1200,0,4,False,True
warehouse,object,1200,0,3,False,True
order_status,object,1200,0,3,False,True
units_sold,int64,1200,0,683,True,False
inventory_level,int64,1200,0,1069,True,False
transportation_cost,float64,1200,0,1200,True,False
order_accuracy,bool,1200,0,2,False,False


In [42]:
#fix date issue
inventory['date'] = pd.to_datetime(inventory['date'], format="%d-%m-%Y", errors='coerce')

In [45]:
inventory.head(25)


,date,region,category,supplier,warehouse,order_status,units_sold,inventory_level,transportation_cost,order_accuracy,lead_time_days,backorder,cost_of_goods_sold_cogs,average_inventory,warehouse_capacity
0,2020-01-14,North,Accessories,Supplier A,Warehouse 1,Fulfilled,302,2124,1103.838324,True,9,False,37820.052400,2048.0,5037
1,2020-11-15,East,Furniture,Supplier D,Warehouse 2,Fulfilled,741,1972,13163.007660,True,11,False,54396.173690,1213.0,9216
2,2020-04-16,East,Furniture,Supplier C,Warehouse 2,Fulfilled,940,454,9872.294126,True,17,False,24217.454620,1160.5,7699
3,2020-04-17,South,Accessories,Supplier D,Warehouse 2,Canceled,589,1867,4547.589932,True,3,False,38919.080050,3364.5,9271
4,2020-04-18,North,Accessories,Supplier C,Warehouse 2,Fulfilled,964,4862,11994.042310,True,21,False,59204.506830,3063.0,5828
5,2020-04-19,South,Electronics,Supplier A,Warehouse 1,Canceled,338,1264,6220.991375,False,15,False,55736.887440,1104.5,8292
6,2020-04-20,West,Furniture,Supplier A,Warehouse 2,Pending,748,945,14023.275560,True,7,False,139260.837700,929.0,5073
7,2020-04-21,North,Accessories,Supplier C,Warehouse 1,Canceled,909,913,1556.256814,True,25,False,35640.521770,2204.0,9860
8,2020-04-22,East,Clothing,Supplier C,Warehouse 3,Fulfilled,781,3495,4933.463849,True,8,False,73883.090710,3481.0,8375
9,2020-04-23,East,Clothing,Supplier A,Warehouse 1,Fulfilled,488,3467,12041.493800,True,21,False,52012.689620,4067.5,8594


In [49]:
# Round both columns to 2 decimal places
inventory["transportation_cost"] = inventory["transportation_cost"].round(2)
inventory["cost_of_goods_sold_cogs"] = inventory["cost_of_goods_sold_cogs"].round(2)

In [50]:
inventory.head()

,date,region,category,supplier,warehouse,order_status,units_sold,inventory_level,transportation_cost,order_accuracy,lead_time_days,backorder,cost_of_goods_sold_cogs,average_inventory,warehouse_capacity
0,2020-01-14,North,Accessories,Supplier A,Warehouse 1,Fulfilled,302,2124,1103.84,True,9,False,37820.05,2048.0,5037
1,2020-11-15,East,Furniture,Supplier D,Warehouse 2,Fulfilled,741,1972,13163.01,True,11,False,54396.17,1213.0,9216
2,2020-04-16,East,Furniture,Supplier C,Warehouse 2,Fulfilled,940,454,9872.29,True,17,False,24217.45,1160.5,7699
3,2020-04-17,South,Accessories,Supplier D,Warehouse 2,Canceled,589,1867,4547.59,True,3,False,38919.08,3364.5,9271
4,2020-04-18,North,Accessories,Supplier C,Warehouse 2,Fulfilled,964,4862,11994.04,True,21,False,59204.51,3063.0,5828


In [51]:
# choose numeric columns
num_cols = inventory.select_dtypes(include="number").columns.tolist()

rows = []
bounds_lo = {}
bounds_hi = {}

for c in num_cols:
    q1 = inventory[c].quantile(0.25)
    q3 = inventory[c].quantile(0.75)
    iqr = q3 - q1
    lo = q1 - 1.5 * iqr
    hi = q3 + 1.5 * iqr
    bounds_lo[c] = lo
    bounds_hi[c] = hi
    mask = (inventory[c] < lo) | (inventory[c] > hi)
    rows.append({
        "column": c,
        "q1": q1, "q3": q3, "iqr": iqr,
        "lower_bound": lo, "upper_bound": hi,
        "outliers": int(mask.sum()),
        "outlier_pct": round(100 * mask.mean(), 2)
    })

iqr_summary = pd.DataFrame(rows).sort_values("outlier_pct", ascending=False)
iqr_summary

,column,q1,q3,iqr,lower_bound,upper_bound,outliers,outlier_pct
4,cost_of_goods_sold_cogs,22982.3700,88006.5475,65024.1775,-74553.89625,185542.81375,6,0.5
1,inventory_level,1261.7500,3726.2500,2464.5000,-2435.00000,7423.00000,0,0.0
0,units_sold,298.0000,777.0000,479.0000,-420.50000,1495.50000,0,0.0
2,transportation_cost,4145.8825,11154.0200,7008.1375,-6366.32375,21666.22625,0,0.0
3,lead_time_days,8.0000,23.0000,15.0000,-14.50000,45.50000,0,0.0
5,average_inventory,1843.0000,3240.1250,1397.1250,-252.68750,5335.81250,0,0.0
6,warehouse_capacity,6116.5000,8662.2500,2545.7500,2297.87500,12480.87500,0,0.0


In [53]:
#standardize the order status
inventory["order_status"] = inventory["order_status"].str.strip().str.lower()

In [54]:
inventory.head()


,date,region,category,supplier,warehouse,order_status,units_sold,inventory_level,transportation_cost,order_accuracy,lead_time_days,backorder,cost_of_goods_sold_cogs,average_inventory,warehouse_capacity,any_iqr_outlier
0,2020-01-14,North,Accessories,Supplier A,Warehouse 1,fulfilled,302,2124,1103.84,True,9,False,37820.05,2048.0,5037,False
1,2020-11-15,East,Furniture,Supplier D,Warehouse 2,fulfilled,741,1972,13163.01,True,11,False,54396.17,1213.0,9216,False
2,2020-04-16,East,Furniture,Supplier C,Warehouse 2,fulfilled,940,454,9872.29,True,17,False,24217.45,1160.5,7699,False
3,2020-04-17,South,Accessories,Supplier D,Warehouse 2,canceled,589,1867,4547.59,True,3,False,38919.08,3364.5,9271,False
4,2020-04-18,North,Accessories,Supplier C,Warehouse 2,fulfilled,964,4862,11994.04,True,21,False,59204.51,3063.0,5828,False


Now that the we have cleaned that data, we will derive dimension and fact tables from them.

In [55]:
date_col = "date" if "date" in inventory.columns else "Date"

inventory[date_col] = pd.to_datetime(inventory[date_col], errors="coerce")
assert inventory[date_col].notna().any(), "No valid dates found after parsing."

cal = (
    pd.Series(sorted(inventory[date_col].dropna().dt.normalize().unique()), name="date")
    .to_frame()
)

#Core attributes
cal["date_key"]      = cal["date"].dt.strftime("%Y%m%d").astype(int)   # surrogate key
cal["year"]          = cal["date"].dt.year
cal["quarter"]       = cal["date"].dt.quarter
cal["month"]         = cal["date"].dt.month
cal["month_name"]    = cal["date"].dt.month_name()
cal["day"]           = cal["date"].dt.day
cal["day_name"]      = cal["date"].dt.day_name()
cal["week_iso"]      = cal["date"].dt.isocalendar().week.astype(int)
cal["year_iso"]      = cal["date"].dt.isocalendar().year.astype(int)
cal["week_start"]    = (cal["date"] - pd.to_timedelta(cal["date"].dt.weekday, unit="D"))
cal["week_end"]      = cal["week_start"] + pd.to_timedelta(6, unit="D")
cal["is_weekend"]    = cal["date"].dt.dayofweek.isin([5, 6])

#Helpful time buckets
cal["year_quarter"]  = cal["year"].astype(str) + " Q" + cal["quarter"].astype(str)
cal["year_month"]    = cal["date"].dt.strftime("%Y-%m")

#Final dim_date (ordered)
dim_date = cal[
    [
        "date_key", "date",
        "year", "quarter", "year_quarter",
        "month", "month_name", "year_month",
        "week_iso", "year_iso", "week_start", "week_end",
        "day", "day_name", "is_weekend"
    ]
].sort_values("date").reset_index(drop=True)



In [56]:
dim_date.head()

,date_key,date,year,quarter,year_quarter,month,month_name,year_month,week_iso,year_iso,week_start,week_end,day,day_name,is_weekend
0,20200114,2020-01-14,2020,1,2020 Q1,1,January,2020-01,3,2020,2020-01-13,2020-01-19,14,Tuesday,False
1,20200416,2020-04-16,2020,2,2020 Q2,4,April,2020-04,16,2020,2020-04-13,2020-04-19,16,Thursday,False
2,20200417,2020-04-17,2020,2,2020 Q2,4,April,2020-04,16,2020,2020-04-13,2020-04-19,17,Friday,False
3,20200418,2020-04-18,2020,2,2020 Q2,4,April,2020-04,16,2020,2020-04-13,2020-04-19,18,Saturday,True
4,20200419,2020-04-19,2020,2,2020 Q2,4,April,2020-04,16,2020,2020-04-13,2020-04-19,19,Sunday,True


In [57]:
#Download the cleaned data (70% cleaned) 
def create_download_link(dim_date, filename="dim_date.csv"):
    csv = dim_date.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return HTML(f'<a download="{filename}" href="data:text/csv;base64,{b64}">Download CSV</a>')

create_download_link(dim_date)

Now we work on Category

In [61]:
#Get unique categories
categories = inventory["category"].dropna().unique()

#Build a DataFrame for dim_category
dim_category = pd.DataFrame({
    "category_id": ["CA" + str(i+1).zfill(3) for i in range(len(categories))],
    "category_name": categories
})


In [65]:
dim_category.head()

,category_id,category_name
0,CA001,Accessories
1,CA002,Furniture
2,CA003,Electronics
3,CA004,Clothing


In [66]:
#Download the cleaned data (70% cleaned) 
def create_download_link(dim_category, filename="dim_category.csv"):
    csv = dim_category.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return HTML(f'<a download="{filename}" href="data:text/csv;base64,{b64}">Download CSV</a>')

create_download_link(dim_category)

In [71]:
#Get unique supplier names
suppliers = sorted(inventory["supplier"].dropna().unique())

#Build dim_supplier
dim_supplier = pd.DataFrame({
    "supplier_id": ["SU" + str(i+1).zfill(3) for i in range(len(suppliers))],
    "supplier_name": suppliers
})


In [72]:
dim_supplier.head(10)

,supplier_id,supplier_name
0,SU001,Supplier A
1,SU002,Supplier B
2,SU003,Supplier C
3,SU004,Supplier D


In [73]:
#Download the cleaned data (70% cleaned) 
def create_download_link(dim_supplier, filename="dim_supplier.csv"):
    csv = dim_supplier.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return HTML(f'<a download="{filename}" href="data:text/csv;base64,{b64}">Download CSV</a>')

create_download_link(dim_supplier)

In [75]:
#Get unique regions and sort alphabetically
regions = sorted(inventory["region"].dropna().unique())

#Build dim_region
dim_region = pd.DataFrame({
    "region_id": ["R" + str(i+1).zfill(3) for i in range(len(regions))],
    "region_name": regions
})


In [76]:
dim_region.head()

,region_id,region_name
0,R001,East
1,R002,North
2,R003,South
3,R004,West


In [96]:
#Download the cleaned data (70% cleaned) 
def create_download_link(dim_region, filename="dim_region.csv"):
    csv = dim_region.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return HTML(f'<a download="{filename}" href="data:text/csv;base64,{b64}">Download CSV</a>')

create_download_link(dim_region)

Now for Warehouse

In [87]:
#Sort by both for stable IDs
warehouses = (
    inventory[["warehouse", "region"]]
    .drop_duplicates()
    .sort_values(["warehouse", "region"])
    .reset_index(drop=True)
)

#Map region -> region_id
region_map = dict(zip(dim_region["region_name"], dim_region["region_id"]))
warehouses["region_id"] = warehouses["region"].map(region_map)

#Sanity checks
assert warehouses["region_id"].notna().all(), "Unmapped region found."
assert warehouses.duplicated(["warehouse", "region"]).sum() == 0, "Duplicate warehouse x region."

#Build dim_warehouse
dim_warehouse = pd.DataFrame({
    "warehouse_id": ["W" + str(i+1).zfill(3) for i in range(len(warehouses))],
    "warehouse_name": warehouses["warehouse"].values,
    "region_id": warehouses["region_id"].values
}).sort_values(["warehouse_name", "region_id"]).reset_index(drop=True)



In [89]:
dim_warehouse.head(20)

,warehouse_id,warehouse_name,region_id
0,W001,Warehouse 1,R001
1,W002,Warehouse 1,R002
2,W003,Warehouse 1,R003
3,W004,Warehouse 1,R004
4,W005,Warehouse 2,R001
5,W006,Warehouse 2,R002
6,W007,Warehouse 2,R003
7,W008,Warehouse 2,R004
8,W009,Warehouse 3,R001
9,W010,Warehouse 3,R002


In [90]:
#Download the cleaned data (70% cleaned) 
def create_download_link(dim_warehouse, filename="dim_warehouse.csv"):
    csv = dim_warehouse.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return HTML(f'<a download="{filename}" href="data:text/csv;base64,{b64}">Download CSV</a>')

create_download_link(dim_warehouse)

Now we build the fact table

In [100]:
# 0) Normalize column names
fact = inventory.copy()
fact.columns = [c.strip().lower().replace(" ", "_") for c in fact.columns]

# 1) Date + date_key
fact["date"] = pd.to_datetime(fact["date"], errors="coerce")
fact["date_key"] = fact["date"].dt.strftime("%Y%m%d").astype("Int64")

# 2) Normalize join columns
for col in ["warehouse", "region", "category", "supplier", "order_status"]:
    if col in fact.columns:
        fact[col] = fact[col].astype(str).str.strip()

# 3) Attach region_id from dim_region, then drop region text
dim_region_norm = dim_region.copy()
dim_region_norm["region_name"] = dim_region_norm["region_name"].astype(str).str.strip()

fact = fact.merge(
    dim_region_norm.rename(columns={"region_name": "region"})[["region", "region_id"]],
    on="region",
    how="left",
    validate="many_to_one"
).drop(columns=["region"])

# 4) Join warehouse_id using warehouse + region_id
dim_wh = dim_warehouse.rename(columns={"warehouse_name": "warehouse"}).copy()
dim_wh["warehouse"] = dim_wh["warehouse"].astype(str).str.strip()

fact = fact.merge(
    dim_wh[["warehouse_id", "warehouse", "region_id"]],
    on=["warehouse", "region_id"],
    how="left",
    validate="many_to_one"
).drop(columns=["warehouse"])

# 5) Join category_id
dim_cat = dim_category.copy()
dim_cat["category_name"] = dim_cat["category_name"].astype(str).str.strip()
fact["category_match"] = fact["category"].str.lower()
dim_cat["category_match"] = dim_cat["category_name"].str.lower()

fact = fact.merge(
    dim_cat[["category_id", "category_match"]],
    on="category_match",
    how="left",
    validate="many_to_one"
).drop(columns=["category_match", "category"])

# 6) Join supplier_id
dim_sup = dim_supplier.copy()
dim_sup["supplier_name"] = dim_sup["supplier_name"].astype(str).str.strip()
fact["supplier_match"] = fact["supplier"].str.lower()
dim_sup["supplier_match"] = dim_sup["supplier_name"].str.lower()

fact = fact.merge(
    dim_sup[["supplier_id", "supplier_match"]],
    on="supplier_match",
    how="left",
    validate="many_to_one"
).drop(columns=["supplier_match", "supplier"])

# 7) Select final columns (note: only region_id kept)
keep_cols = [
    # Keys
    "date_key", "warehouse_id", "category_id", "supplier_id", "region_id",
    # Measures
    "units_sold", "inventory_level", "average_inventory",
    "cost_of_goods_sold_cogs", "transportation_cost", "lead_time_days",
    # Attributes kept in fact
    "order_status", "backorder", "order_accuracy",
    # Convenience
    "date"
]
keep_cols = [c for c in keep_cols if c in fact.columns]
fact_sales_inventory = fact[keep_cols].copy()

# 8) Sanity checks
for k in ["date_key", "warehouse_id", "category_id", "supplier_id", "region_id"]:
    if k in fact_sales_inventory.columns:
        miss = fact_sales_inventory[k].isna().sum()
        if miss:
            print(f"WARNING: {k} has {miss} missing values")

num_cols = [
    "units_sold", "inventory_level", "average_inventory",
    "cost_of_goods_sold_cogs", "transportation_cost", "lead_time_days"
]
for c in [c for c in num_cols if c in fact_sales_inventory.columns]:
    fact_sales_inventory[c] = pd.to_numeric(fact_sales_inventory[c], errors="coerce")

# === Add warehouse_capacity back (append-only) ===
tmp = inventory.copy()
tmp.columns = [c.strip().lower().replace(" ", "_") for c in tmp.columns]
tmp["warehouse"] = tmp["warehouse"].astype(str).str.strip()
tmp["region"] = tmp["region"].astype(str).str.strip()
tmp["warehouse_capacity"] = pd.to_numeric(tmp["warehouse_capacity"], errors="coerce")

reg_map = dict(zip(
    dim_region["region_name"].astype(str).str.strip(),
    dim_region["region_id"]
))
tmp["region_id"] = tmp["region"].map(reg_map)

wh_cap = (
    tmp[["warehouse", "region_id", "warehouse_capacity"]]
      .dropna(subset=["warehouse_capacity"])
      .groupby(["warehouse", "region_id"], as_index=False)["warehouse_capacity"]
      .agg(lambda s: s.mode().iat[0] if not s.mode().empty else s.median())
)

dim_wh_cap = (
    dim_warehouse.rename(columns={"warehouse_name": "warehouse"})
      .merge(wh_cap, on=["warehouse", "region_id"], how="left")
      [["warehouse_id", "warehouse_capacity"]]
)

fact_sales_inventory = fact_sales_inventory.merge(
    dim_wh_cap, on="warehouse_id", how="left"
)


In [101]:
fact_sales_inventory.head()

,date_key,warehouse_id,category_id,supplier_id,region_id,units_sold,inventory_level,average_inventory,cost_of_goods_sold_cogs,transportation_cost,lead_time_days,order_status,backorder,order_accuracy,date,warehouse_capacity
0,20200114,W002,CA001,SU001,R002,302,2124,2048.0,37820.05,1103.84,9,fulfilled,False,True,2020-01-14,5037
1,20201115,W005,CA002,SU004,R001,741,1972,1213.0,54396.17,13163.01,11,fulfilled,False,True,2020-11-15,5027
2,20200416,W005,CA002,SU003,R001,940,454,1160.5,24217.45,9872.29,17,fulfilled,False,True,2020-04-16,5027
3,20200417,W007,CA001,SU004,R003,589,1867,3364.5,38919.08,4547.59,3,canceled,False,True,2020-04-17,9754
4,20200418,W006,CA001,SU003,R002,964,4862,3063.0,59204.51,11994.04,21,fulfilled,False,True,2020-04-18,6340


In [102]:
cols = list(fact_sales_inventory.columns)

if "warehouse_capacity" in cols and "date" in cols:
    cols.remove("warehouse_capacity")
    insert_at = cols.index("date")
    cols.insert(insert_at, "warehouse_capacity")
    fact_sales_inventory = fact_sales_inventory[cols]
else:
    print("Note: 'warehouse_capacity' or 'date' not found in fact_sales_inventory columns.")


In [103]:
fact_sales_inventory.head()

,date_key,warehouse_id,category_id,supplier_id,region_id,units_sold,inventory_level,average_inventory,cost_of_goods_sold_cogs,transportation_cost,lead_time_days,order_status,backorder,order_accuracy,warehouse_capacity,date
0,20200114,W002,CA001,SU001,R002,302,2124,2048.0,37820.05,1103.84,9,fulfilled,False,True,5037,2020-01-14
1,20201115,W005,CA002,SU004,R001,741,1972,1213.0,54396.17,13163.01,11,fulfilled,False,True,5027,2020-11-15
2,20200416,W005,CA002,SU003,R001,940,454,1160.5,24217.45,9872.29,17,fulfilled,False,True,5027,2020-04-16
3,20200417,W007,CA001,SU004,R003,589,1867,3364.5,38919.08,4547.59,3,canceled,False,True,9754,2020-04-17
4,20200418,W006,CA001,SU003,R002,964,4862,3063.0,59204.51,11994.04,21,fulfilled,False,True,6340,2020-04-18


In [104]:
#Download the cleaned data (70% cleaned) 
def create_download_link(fact_sales_inventory, filename="fact_sales_inventory.csv"):
    csv = fact_sales_inventory.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return HTML(f'<a download="{filename}" href="data:text/csv;base64,{b64}">Download CSV</a>')

create_download_link(fact_sales_inventory)